In [1]:
import numpy as np
import itertools
import os
import sys
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.tools.xsecComputer import XSecComputer, LO
from smodels.tools.physicsUnits import TeV

In [2]:
mHSCP = np.linspace(25.,3000.,100)

In [3]:
# columns = ('msl3','Wsl3')
columns = ['msl3']
outputFile = 'THSCPM1b_pts_validation.txt'
header = '%19s'*len(columns) %tuple(columns)
header = header[3:]
np.savetxt(outputFile,mHSCP,header=header,fmt = ['     %1.7e']*len(columns))

In [4]:
template = 'THSCPM1b_template.slha'
with open(template,'r') as f:
    lines = f.readlines()
iw = [i for i,l in enumerate(lines) if 'DECAY 1000015 1.000000e-30 # Wsl3' in l][0]
im = [i for i,l in enumerate(lines) if '1000015 1.00000000E+02 # Msl3' in l][0]


widths = np.logspace(-25,-15,30)
computer = XSecComputer( LO, 1000, pythiaVersion=8, maycompile=False)
for mass in mHSCP:
    slhaFile = './slha/THSCPM1b_%1.0f.slha' %(mass)
    lines[im] = '  1000015 %1.8E # Msl3 \n' %mass
    with open(slhaFile,'w') as f:
        for l in lines: f.write(l)
            
    xsecs = computer.compute( 8*TeV, slhaFile, unlink=True,
                                    loFromSlha=False, pythiacard= './pythia8_stau.cfg')
    add = computer.addXSecToFile(xsecs,slhaFile)

    for width in widths:
        with open(slhaFile,'r') as ff:
            newLines = ff.readlines()
        
        newFile = './slha/THSCPM1b_%1.0f_%1.2e.slha' %(mass,width)
        newLines[iw] = 'DECAY 1000015 %1.6e # Wsl3 \n' %width
        with open(newFile,'w') as f:
            for l in newLines: f.write(l)
    os.remove(slhaFile)